In [1]:
import os
import time
import copy
import optuna

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models, datasets, transforms

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print(torch.cuda.is_available())

PyTorch Version:  2.6.0+cpu
Torchvision Version:  0.21.0+cpu
False


input dir

In [2]:
CURRENT_DIR = os.getcwd()
MAIN_FOLDER = Path(CURRENT_DIR).parent
OUTPUT_FOLDER = os.path.join(MAIN_FOLDER, 'aligned')  
FOLD_DATA = os.path.join(MAIN_FOLDER, 'fold_data') 

BATCH_SIZE = 32
NUM_CLASSES = 1

DEVICE = torch.device("cuda")


Data processing

In [3]:
# Data Transforms
def get_data_transforms():
    normalize = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

    return {
        'train': transforms.Compose([transforms.ToTensor(), transforms.Normalize(*normalize)]),
        'val': transforms.Compose([transforms.ToTensor(), transforms.Normalize(*normalize)]),
    }

data_transforms = get_data_transforms()

In [4]:
def load_folds_dataset(image_root, fold_dir, fold_files):
    image_paths = []
    labels = []

    for fold_file in fold_files:
        print(f"Reading fold file: {fold_file}")
        with open(os.path.join(fold_dir, fold_file), 'r') as f:
            next(f)  
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) < 5:
                    continue
                user_id = parts[0]
                original_img_name = parts[1]
                gender = parts[4].lower()

                if gender not in ["m", "f"]:
                    continue
                label = 0 if gender == "m" else 1

                user_folder = os.path.join(image_root, user_id)
                if not os.path.isdir(user_folder):
                    continue

                for file in os.listdir(user_folder):
                    if original_img_name in file:
                        full_path = os.path.join(user_folder, file)
                        if os.path.isfile(full_path):
                            image_paths.append(full_path)
                            labels.append(label)
                        break

    return image_paths, labels


In [5]:
class BasicImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[idx], dtype=torch.float32)


In [6]:
def get_dataloaders(batch_size, train_folds, val_fold):
    train_image_paths, train_labels = load_folds_dataset(OUTPUT_FOLDER, FOLD_DATA, train_folds)
    val_image_paths, val_labels = load_folds_dataset(OUTPUT_FOLDER, FOLD_DATA, [val_fold])

    train_dataset = BasicImageDataset(train_image_paths, train_labels, transform=data_transforms['train'])
    val_dataset = BasicImageDataset(val_image_paths, val_labels, transform=data_transforms['val'])

    print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

    if len(train_dataset) == 0 or len(val_dataset) == 0:
        return None

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return {'train': train_loader, 'val': val_loader}


Train

In [7]:
# intialize model
def load_model(num_classes):
    model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model.num_classes = num_classes

    for param in model.parameters():
        param.requires_grad = True

    return model.to(DEVICE)

In [8]:
def train_model(model, dataloaders, optimizer, num_epochs=25, patience=5, trial=None):
    criterion = nn.BCELoss()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0
    epochs_no_improve = 0
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'train_prec': [], 'val_prec': [],
        'train_rec': [], 'val_rec': [],
        'train_f1': [], 'val_f1': [],
    }

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0
            all_preds, all_labels = [], []

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = torch.sigmoid(outputs.squeeze())
                    loss = criterion(outputs, labels)
                    preds = (outputs > 0.5).long()

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(all_labels, all_preds)
            epoch_prec = precision_score(all_labels, all_preds, zero_division=0)
            epoch_rec = recall_score(all_labels, all_preds, zero_division=0)
            epoch_f1 = f1_score(all_labels, all_preds, zero_division=0)

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc)
            history[f'{phase}_prec'].append(epoch_prec)
            history[f'{phase}_rec'].append(epoch_rec)
            history[f'{phase}_f1'].append(epoch_f1)

            print(f"{phase.upper()} — Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} | "
                  f"Prec: {epoch_prec:.4f} | Rec: {epoch_rec:.4f} | F1: {epoch_f1:.4f}")

            if phase == 'val':
                if epoch_f1 > best_f1:
                    best_f1 = epoch_f1
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    model.load_state_dict(best_model_wts)
    print(f"\nTraining complete — Best Val F1: {best_f1:.4f}")
    return model, history

In [9]:
def objective(trial):
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])

    best_val_acc = 0.0
    all_folds = [f"fold_{i}_data.txt" for i in range(5)]

    for fold_idx in range(5):
        val_fold = all_folds[fold_idx]
        train_folds = [f for i, f in enumerate(all_folds) if i != fold_idx]

        print(f"Fold {fold_idx}: Val = {val_fold}, Train = {train_folds}")
        dataloaders = get_dataloaders(batch_size, train_folds, val_fold)

        if dataloaders is None:
            print(f"Skipping fold {fold_idx} due to empty dataset.")
            continue

        model = load_model(NUM_CLASSES)

        params_to_update = [p for p in model.parameters() if p.requires_grad]
        optimizer = optim.Adam(params_to_update, lr=lr, weight_decay=weight_decay)

        model, history = train_model(model, dataloaders, optimizer, num_epochs=50, trial=trial)

        best_val_acc = max(best_val_acc, max(history['val_acc']))

    return best_val_acc


In [10]:
# Hyperparameter Optimization Optuna
study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=20)

print("Best trial:")
trial = study.best_trial
print(f"  Val Accuracy: {trial.value:.4f}")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-03-31 17:28:13,121] A new study created in memory with name: no-name-75dbf638-9321-48a5-8c74-1556903545ee


Fold 0: Val = fold_0_data.txt, Train = ['fold_1_data.txt', 'fold_2_data.txt', 'fold_3_data.txt', 'fold_4_data.txt']
Reading fold file: fold_1_data.txt
Reading fold file: fold_2_data.txt
Reading fold file: fold_3_data.txt
Reading fold file: fold_4_data.txt
Reading fold file: fold_0_data.txt
Train size: 13497, Val size: 3995

Epoch 1/50
TRAIN — Loss: 0.6568 | Acc: 0.6057 | Prec: 0.5910 | Rec: 0.9197 | F1: 0.7196
VAL — Loss: 0.6410 | Acc: 0.6458 | Prec: 0.5971 | Rec: 0.8414 | F1: 0.6985

Epoch 2/50
TRAIN — Loss: 0.6073 | Acc: 0.6881 | Prec: 0.6821 | Rec: 0.8109 | F1: 0.7409
VAL — Loss: 0.6027 | Acc: 0.6809 | Prec: 0.6455 | Rec: 0.7664 | F1: 0.7008

Epoch 3/50
TRAIN — Loss: 0.5712 | Acc: 0.7175 | Prec: 0.7321 | Rec: 0.7671 | F1: 0.7492
VAL — Loss: 0.5801 | Acc: 0.7021 | Prec: 0.6815 | Rec: 0.7305 | F1: 0.7052

Epoch 4/50
TRAIN — Loss: 0.5434 | Acc: 0.7376 | Prec: 0.7588 | Rec: 0.7667 | F1: 0.7627
VAL — Loss: 0.5703 | Acc: 0.7099 | Prec: 0.6887 | Rec: 0.7392 | F1: 0.7130

Epoch 5/50
TRAIN —

In [ ]:
# # Save the model
# torch.save(model, 'efficientnet_test.pth')
# print("Model saved as efficientnet_test.pth")